In [1]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import json
import pandas as pd
from datasets import load_dataset, load_from_disk

In [2]:
!rm -rf /kaggle/working/logs /kaggle/working/results /kaggle/working/wandb

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
# Load model directly
cache_dir = "/kaggle/working/my_model_dir"
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)

In [5]:
cache_dir = "/kaggle/working/my_data_dir"

ds = load_dataset("Kyudan/MathBridge",  cache_dir=cache_dir)

Generating train split:   0%|          | 0/23195831 [00:00<?, ? examples/s]

In [6]:
ds_train = ds["train"]

In [7]:
def preprocess_data(examples):
    before = examples["context_before"]
    after = examples["context_after"]
    equation = examples["equation"]
    spoken_English = examples["spoken_English"]

    # Prepend a task-specific prompt if necessary, e.g., "translate English to LaTeX:"
    inputs = [f"translate English to LaTeX: {before} {spoken_English} {after}"]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

    outputs = [f"{before} {equation} {after}"]
    
    with tokenizer.as_target_tokenizer():
        model_outputs = tokenizer(outputs, max_length=512, truncation=True, padding=True)

    model_inputs["labels"] = model_outputs["input_ids"]

    return model_inputs

In [8]:
ds_train = ds_train.filter(lambda x: len(x["equation"])>10)
ds_train

Filter:   0%|          | 0/23195831 [00:00<?, ? examples/s]

Dataset({
    features: ['context_before', 'equation', 'context_after', 'spoken_English'],
    num_rows: 9458549
})

In [9]:
ds_train_preprocessed = (ds_train.shuffle(seed=42)
                                 .select(range(2*10**5))
                                 .map(preprocess_data, remove_columns=ds_train.column_names, batched=True, batch_size=4))
ds_train_preprocessed

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})

In [10]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to=None,
    run_name="T5_Finetune"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train_preprocessed,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [11]:
import wandb
wandb.login(key="91a0db028dce6f175361702b5140fa9c941bf8ff")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
trainer.train()

wandb: Currently logged in as: deeptanshumalu (deeptanshu-malu). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240915_132836-1byivhu3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run T5_Finetune
wandb: ⭐️ View project at https://wandb.ai/deeptanshu-malu/huggingface
wandb: 🚀 View run at https://wandb.ai/deeptanshu-malu/huggingface/runs/1byivhu3
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimensi

Step,Training Loss
3125,0.434700
6250,0.175000
9375,0.158800


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=9375, training_loss=0.25619255208333336, metrics={'train_runtime': 10677.8593, 'train_samples_per_second': 14.048, 'train_steps_per_second': 0.878, 'total_flos': 3.573262140162048e+16, 'train_loss': 0.25619255208333336, 'epoch': 3.0})

In [13]:
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json',
 './fine_tuned_t5/tokenizer.json')